This is a note how to start a [data algebra](https://github.com/WinVector/data_algebra) pipeline with user supplied SQL.
The idea is: in addition to starting query work from tables and materialzied views, one can
start a calculation from a SQL query.

Let's show how this works.

We import our libraries and set up some example data which we push into
our notional example database.

In [1]:
import pandas

from data_algebra.data_ops import *
import data_algebra.SQLite
import data_algebra.test_util

In [2]:
d1 = pandas.DataFrame({
    'g': ['a', 'a', 'b', 'b'],
    'v1': [1, 2, 3, 4],
    'v2': [5, 6, 7, 8],
})

In [3]:
sqlite_handle = data_algebra.SQLite.example_handle()
sqlite_handle.insert_table(d1, table_name='d1')

(TableDescription(table_name="d1", column_names=["g", "v1", "v2"]))

Now suppose we have a "by hand" or "user SQL" query we want to start
with.

In [4]:
user_sql = """
  SELECT
    *,
    v1 * v2 AS v3
  FROM
    d1
"""

The idea is: we pretend we have a table with the columns `[v1, v2, v3]` and
realize that by starting our data algebra pipeline with the query instead
of a table reference or description.

For example, if we wanted to create a pipeline that incorporates
the user supplied SQL and creates a new column `v4 := v3 + v1`, we
would do it as so.

In [5]:
ops = (
    SQLNode(
        sql=user_sql,
        column_names=['g', 'v1', 'v2', 'v3'],
        view_name='derived_results'
        )
        .extend({'v4': 'v3 + v1'})
)

This entire combined pipeline can be translated into SQL.

In [6]:
print(sqlite_handle.to_sql(ops))

-- data_algebra SQL https://github.com/WinVector/data_algebra
--  dialect: SQLiteModel 1.4.3
--       string quote: '
--   identifier quote: "
WITH
 "derived_results" AS (
  -- user supplied SQL
     SELECT
       *,
       v1 * v2 AS v3
     FROM
       d1
 )
SELECT  -- .extend({ 'v4': 'v3 + v1'})
 "g" ,
 "v1" ,
 "v2" ,
 "v3" ,
 "v3" + "v1" AS "v4"
FROM
 "derived_results"



And it can, of course, be executed in the database.

In [7]:
res_sqllite = sqlite_handle.read_query(ops)

res_sqllite

,g,v1,v2,v3,v4
0,a,1,5,5,6
1,a,2,6,12,14
2,b,3,7,21,24
3,b,4,8,32,36


The returned table matches our expectations.

In [8]:
expect = d1.copy()
expect['v3'] = expect['v1'] * expect['v2']
expect['v4'] = expect['v3'] + expect['v1']

assert data_algebra.test_util.equivalent_frames(res_sqllite, expect)

We can also run the same query in Pandas if we supply a database handle with tables satisfying the user SQL query requirements..

In [9]:
dr = d1.copy()
dr['v3'] = dr['v1'] * dr['v2']

res_pandas = ops.eval({'derived_results': sqlite_handle})

res_pandas

,g,v1,v2,v3,v4
0,a,1,5,5,6
1,a,2,6,12,14
2,b,3,7,21,24
3,b,4,8,32,36


In [10]:
assert data_algebra.test_util.equivalent_frames(res_pandas, expect)

And there we have it: starting data algebra pipelines from arbitrary SQL instead of
merely from a table reference.

In [11]:
sqlite_handle.close()
